# DataPaser

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import sys

In [2]:
f = open("dataset.csv", "r", errors='ignore')
f.seek(0, 2)
end_position = f.tell()
f.seek(0, 0)

0

In [3]:
a=f.readline()

In [4]:
header = [x for x in  a.replace('"', "").replace('\n', "").split(",") if len(x)>0]
df = pd.DataFrame(columns = ('username', 'query_date', 'report_name', 'doc', 'start_date', 'end_date', 'is_period'))
lines = f.readlines()

In [5]:
for idx, raw_data in enumerate(lines):
    if(idx%500==0):
        print("Processing: {0:.2f}%".format(idx*100/len(lines)))
    try:
        parse = [x for x in raw_data.replace(",", "").replace(":", "").replace("}", "").split('"') if len(x)>3]
        if(len(parse)<9):
#             if('downloadWithGuid' in raw_data or 'downloadWithIndex' in raw_data):
#                 continue
#             if('at' in raw_data):
#                 continue
#             else:
#                 #print(raw_data)
            continue
        d = dict()
        if(parse[3]=='queryWithIndex'):
            d["username"] = parse[1]
            d["query_date"] = datetime.strptime(parse[2], '%d-%m_ -%y')
            d["report_name"] = parse[4]
            assert d["username"]==d["report_name"], "Name different!"
            if('startDate' in parse):
                d["is_period"] = True
                startDate_index = parse.index('startDate')
                docs = parse[5:startDate_index]
                assert len(docs)>0, 'doc list length error'
                assert parse[startDate_index]== 'startDate', " 'startDate' format error"
                if(parse[startDate_index+1]!='null'):
                    d["start_date"] = pd.Period(datetime.strptime(parse[startDate_index+1], '%b %d %Y %I%M%S %p'), freq='D')
                else:
                    d["start_date"] = float('nan')
                assert parse[startDate_index+2]== 'endDate', " 'endDate' format error"
                if(parse[startDate_index+3]!='null'):
                    d["end_date"] = pd.Period(datetime.strptime(parse[startDate_index+3], '%b %d %Y %I%M%S %p'), freq='D')
                else:
                    d["end_date"] = float('nan')
            else:
                d["is_period"] = False
                d["start_date"] =float('nan')
                d["end_date"] = float('nan')
                doc_end_flag = parse.index(']null')
                docs = parse[5:doc_end_flag]
            for doc in docs:
                d["doc"] = doc
                df = df.append(d, ignore_index=True)
    except:
            print("Unexpected error:", sys.exc_info())
            print(parse)
            print(raw_data)
    

Processing: 0.00%
Processing: 1.01%
Processing: 2.02%
Processing: 3.03%
Processing: 4.03%
Processing: 5.04%
Processing: 6.05%
Processing: 7.06%
Processing: 8.07%
Unexpected error: (<class 'AssertionError'>, AssertionError('doc list length error',), <traceback object at 0x10896be88>)
['10.67.155.140', 'esp.admin', '22-2_ -17', 'queryWithIndex', 'esp.admin', 'startDate', 'null', 'endDate', 'Dec 31 9999 120000 AM', 'conditionList', 'logicalOperator', 'nullnull150]', '2006', '_____IP  10.67.155.140 _']
"10.67.155.140,""esp.admin"",22-2_ -17,""queryWithIndex"",""[""""esp.admin"""",[""""""""],{""""startDate"""":null,""""endDate"""":""""Dec 31, 9999 12:00:00 AM""""},""""Y"""",{""""conditionList"""":[],""""logicalOperator"""":null},null,1,50]"",""2006"",""_____IP : 10.67.155.140 _"",0",

Unexpected error: (<class 'AssertionError'>, AssertionError('doc list length error',), <traceback object at 0x10896bac8>)
['10.67.155.140', 'esp.admin', '22-2_ -17', 'queryWithIndex', 'esp.admin', 'startDate',

Unexpected error: (<class 'AssertionError'>, AssertionError('doc list length error',), <traceback object at 0x1095a4908>)
['10.67.67.52', 'mPOS', '20-4_ -17', 'queryWithIndex', 'mPOS', 'startDate', 'null', 'endDate', 'null', 'conditionList', 'classType', 'Condition', 'clause', 'null', 'condition', 'logicalOperator', 'comparisonOperator', 'reportClass', 'value', 'logicalOperator', 'nullnull1-1]', '3001', '_____table', '304640']
"10.67.67.52,""mPOS"",20-4_ -17,""queryWithIndex"",""[""""mPOS"""",[""""""""],{""""startDate"""":null,""""endDate"""":null},""""Y"""",{""""conditionList"""":[{""""classType"""":""""Condition"""",""""clause"""":null,""""condition"""":{""""logicalOperator"""":""""AND"""",""""comparisonOperator"""":""""EQ"""",""""key"""":""""reportClass"""",""""value"""":""""8""""}}],""""logicalOperator"""":null},null,1,-1]"",""3001"",""_____table"",304640",

Processing: 91.79%
Processing: 92.80%
Processing: 93.81%
Processing: 94.82%
Processing: 95.83%
Processing: 96.83%
Processing:

In [11]:
df.head()

,username,query_date,report_name,doc,start_date,end_date,is_period
0,001713,2017-02-10,001713,ODS_UNB_0460,2016-08-31,2016-08-31,True
1,calvin,2017-02-10,calvin,ODS_PA_0500_P,2016-11-30,2016-11-30,True
2,calvin,2017-02-10,calvin,ODS_FIN_0573,NaT,9999-12-31,True
3,user1,2017-02-10,user1,ODS_POS_0990,NaT,9999-12-31,True
4,user1,2017-02-10,user1,ODS_POS_0990,NaT,9999-12-31,True


In [7]:
df.shape

(32323, 7)

In [8]:
df.to_csv('out.csv')

In [9]:
df['username'].value_counts()

mPOS          19206
CSIS-SIT3      1900
user1           865
user4           667
000442          623
roxana          611
user2           525
CSIS            489
user18          466
yusiang         382
user3           341
user5           335
shiang          330
000768          308
001415          281
002064          258
000358          239
000898          224
000368          212
ricky           206
user10          206
000923          195
calvin          171
000316          161
user6           144
user11          118
000279          118
jesse           111
user8           111
000718          105
              ...  
001414           21
CSIS-R5          20
user16           19
001883           17
AGYYEE           16
001416           14
000217           13
invdaphne        13
NBJENIA          12
test             12
LIFE             10
janus             9
000323            8
000270            8
000551            8
allencha          7
001420            6
000622            6
Cheng             5


In [10]:
df['doc'].value_counts()

report001                           3396
FMT_POS_0130                         758
FMT_POS_0190                         632
FMT_POS_190                          578
#POLICY#                             559
FMT_PA_0750                          556
FMT_POS_0290                         551
FMT_POS_0470                         550
FMT_POS_0360                         535
FMT_POS_0180                         527
FMT_POS_0070                         525
FMT_POS_0080                         523
FMT_POS_0100                         515
FMT_PA_0710                          496
FMT_POS_0150                         493
FMT_POS_0090                         491
FMT_POS_0270                         487
FMT_POS_1670                         484
FMT_POS_0310                         483
FMT_POS_0320                         476
FMT_POS_1570                         473
FMT_POS_7210                         472
FMT_POS_0110                         472
FMT_POS_0340                         471
FMT_POS_7230    